# Sample code scripts for LangChain and LangGraph

github source:
- **[LangChain](https://github.com/langchain-ai/langchain)**
- **[LangGraph](https://github.com/langchain-ai/langgraph)**

LangChain intro: https://docs.langchain.com.cn/docs/introduction/

LangGraph intro and tutorial: https://langgraphcn.org/

## A sample test for LangGraph

reference: https://blog.csdn.net/forevercui/article/details/145876616

这是一个简单的使用 LangGraph 构建 Agent 组件的案例，用于回答用户的问题。它的结构是 
```bash
START -> chatbot -> END
```

该案例中就包含了 LangGraph 的几个基础组件
1. 状态, state: 可以是一个简单的字典或者Pydantic模型。状态包含了应用运行时需要的所有信息以及模型生成的信息;
2. 节点, node: 节点通常是Python函数,用于处理状态并返回更新后的状态;
3. 边, edge: 边定义了节点之间的连接关系和路由逻辑.

In [1]:
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import MessagesState
 
# llm = ChatOpenAI(
#     base_url="https://lxxxxx.enovo.com/v1/", 
#     api_key="sxxxxxxxwW",
#     model_name="qwen2.5-instruct"
#     )

llm = ChatOpenAI(
    base_url="https://api.deepseek.com", 
    api_key="sk-3b458ee0624f41e1b8c589e74be23e44",
    model_name="deepseek-chat"
    )

# 创建节点函数
def chatbot(state: MessagesState):
    return {"messages": [llm.invoke(state["messages"])]}

def build_graph():
    # 创建graph
    graph_builder = StateGraph(MessagesState)
    # 添加起始边
    graph_builder.add_edge(START, "chatbot")
    # 添加节点，命名为chatbot
    graph_builder.add_node("chatbot", chatbot)
    # 添加结束边
    graph_builder.add_edge("chatbot", END)
    # 代码编译为图
    graph = graph_builder.compile()
    return graph


graph = build_graph()
# 生成图的流程图片，写到当前目录下
qa_async_png = graph.get_graph().draw_mermaid_png()
with open("chatbot.png", "wb") as f:
    f.write(qa_async_png)
 
# 流式调用大模型
for event in graph.stream({"messages": [{"role": "user", "content": "中国最好的大学是哪所？"}]}):
    for value in event.values():
        print("Assistant:", value["messages"][-1].content)

Assistant: 中国有多所顶尖大学在不同领域各具优势，以下是几所公认的顶尖高校及其特点：

1. **清华大学**  
   - **优势学科**：工程、计算机科学、材料科学、经济管理等。  
   - **国际排名**：常年位居中国内地高校榜首（如QS、THE等排名）。  
   - **特点**：以理工科见长，科研实力雄厚，被誉为“中国MIT”。

2. **北京大学**  
   - **优势学科**：人文社科、基础科学（数学、物理等）、医学、法学。  
   - **国际排名**：与清华齐名，综合实力顶尖。  
   - **特点**：文理学科均衡，学术自由氛围浓厚，常被比作“中国哈佛”。

3. **复旦大学** & **上海交通大学**  
   - **复旦**：人文社科、医学、自然科学突出，华东地区顶尖综合性大学。  
   - **上交**：工科（船舶、机械等）、医学（附属瑞金医院）、商科强劲。  

4. **浙江大学**  
   - **优势**：学科覆盖全面，农业工程、计算机、光学工程等领域领先。  

5. **中国科学技术大学**  
   - **特色**：理科（物理、化学）和前沿科技（量子信息）研究国际一流，小而精的研究型大学。  

**其他顶尖院校**：  
- **南京大学**（基础学科强）、**中国人民大学**（社科顶尖）、**北京协和医学院**（医学第一）等。  

**选择建议**：  
- 若倾向理工科，清华、中科大、浙大更优；  
- 若偏好人文社科或综合学术，北大、复旦、人大是首选；  
- 医学可考虑北大医学部、协和或上交医学院。  

中国高校的“最好”取决于具体学科和个人发展方向，建议结合专业排名、师资、地理位置等因素综合选择。


我们也可以使用 LangGraph 构建一个 RAG-Agent, RAG(Retrieval-Augmented Generation) 是一种结合了检索和生成两种能力的模型，可以用于处理复杂的问题。它的结构如图

<center><img src="../asserts/langgraph/rag-agent.png"><RAG-Agent Graph></center>

source: https://langgraphcn.org/tutorials/rag/langgraph_agentic_rag/
下面仅给出使用 LangGraph 构建RAG-Agent的图流程的代码，此外我们还需要定义 `agent`, `rewrite`, `generate`, `retrieve` 四个节点函数，以及 `RAGState` 状态类，这些内容在LangGraph的官方文档中都有详细的介绍。

In [2]:
from typing import Annotated, Sequence
from typing_extensions import TypedDict

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages


class AgentState(TypedDict):
    # The add_messages function defines how an update should be processed
    # Default is to replace. add_messages says "append"
    messages: Annotated[Sequence[BaseMessage], add_messages]

In [ ]:
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import ToolNode, tools_condition
from xxx import grade_documents, agent, rewrite, generate, retriever_tool

# Define a new graph
workflow = StateGraph(AgentState)

# Define the nodes we will cycle between
workflow.add_node("agent", agent)  # agent
retrieve = ToolNode([retriever_tool])
workflow.add_node("retrieve", retrieve)  # retrieval
workflow.add_node("rewrite", rewrite)  # Re-writing the question
workflow.add_node(
    "generate", generate
)  # Generating a response after we know the documents are relevant
# Call agent node to decide to retrieve or not
workflow.add_edge(START, "agent")

# Decide whether to retrieve
workflow.add_conditional_edges(
    "agent",
    # Assess agent decision
    tools_condition,
    {
        # Translate the condition outputs to nodes in our graph
        "tools": "retrieve",
        END: END,
    },
)

# Edges taken after the `action` node is called.
workflow.add_conditional_edges(
    "retrieve",
    # Assess agent decision
    grade_documents,
)
workflow.add_edge("generate", END)
workflow.add_edge("rewrite", "agent")

# Compile
graph = workflow.compile()